In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import torchvision
import torchvision.transforms as transforms
torch.set_printoptions(linewidth=120)

In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])
)

In [4]:
# turning off the computation graph 
torch.set_grad_enabled(True)

In [5]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [6]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
        self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=60)
        self.out = nn.Linear(in_features=60,out_features=10)
    def forward(self,t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t,kernel_size=2,stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t,kernel_size=2,stride=2)

        t = F.relu(self.fc1(t.reshape(-1,12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)

        return t

In [11]:
network = Network()
data_loader = torch.utils.data.DataLoader(
    train_set,batch_size=100
)
for epoch in range(5):
    total_loss=0
    total_correct=0
    optimizer = optim.Adam(network.parameters(),lr=0.01)
    for batch in data_loader:
        images, labels = batch
        
        preds = network(images)
        loss = F.cross_entropy(preds,labels)
        
        optimizer.zero_grad() #set gradient to zero
        loss.backward()
        optimizer.step() # update the wieghts

        total_correct+=get_num_correct(preds,labels)
        total_loss+=loss.item()
    print("epoch :",epoch,"Total correct :",total_correct,"Total loss :",total_loss)


epoch : 0 Total correct : 47349 Total loss : 335.961001932621
epoch : 1 Total correct : 51147 Total loss : 238.4395396709442
epoch : 2 Total correct : 51785 Total loss : 220.86038941144943
epoch : 3 Total correct : 52124 Total loss : 214.10244299471378
epoch : 4 Total correct : 52323 Total loss : 207.7776772081852


In [12]:
total_correct/len(train_set)

0.87205

In [13]:
# Confusion Matrix#